## 安裝所需

- Langchain
- Openai



In [ ]:
%pip install langchain openai gradio typing-extensions

### 確認版本
一定要大於1.2.3

In [ ]:
%pip show openai | findstr Version

## 建立opena ai assistant
### assistant api 原理
![openai_assistant](image/openai_assistant.png)


### 方便輸出的CODE


In [ ]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

### 創建Assistant

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="your-api-key")

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="你是一個數學家教，請簡短的以繁體中文解釋並回復",
    model="gpt-4-1106-preview",
)
show_json(assistant)

### 創建thread

In [ ]:
thread = client.beta.threads.create()
show_json(thread)

### add message
將想要知道的訊息加入thread 中

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="我想知道 `3x + 11 = 14`. 請幫我回答",
)
show_json(message)

### Run

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

### Messages
run 完之後代表assistant 已建立，此時可以送api過去

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

可以確認再跑的狀態

In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

也可以問更多問題

In [ ]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="你可以解釋嗎?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)